<img src="images/quantum_algorithms_tutorials.png" alt="drawing" width="100" align="left"/>

<h1 align="center">
Iterative quantum phase estimation
</h1>

The iterative quantum phase estimation (IQPE, also known as IPEA) algorithm is a quantum algorithm for estimating the phase (or eigenvalue) of an eigenvector of a unitary operator [[1](https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm), [2](https://www.tandfonline.com/doi/abs/10.1080/00268976.2011.552441)]. One of the main applications of the algorithm is to estimate eigenvalues (energies) of some molecule's $H$ Hamiltonian. Because $H$ is a Hermitian operator, not unitary, (the algorithm works only with unitary operators) we can't estimate directly its eigenvalues. Instead, we create some unitary operator from $H$ and estimate not the eigenvalues of $H$, but something different (the phase). From the estimated phase one can obtain the corresponding eigenvalue of $H$. So, in the end, we are not only estimating the phase but, what is more important, the desired eigenvalue. Here are the main steps of the algorithm:

1) Create unitary operator $U$ from given $H$: 
$$U = e^{iHt}.$$ 
Here $t$ is a parameter.
   
2) Apply $U$ on the eigenstate $\left| \psi_k \right\rangle$ of $H$. For simplicity, we will assume that the eigenstate is given: 

$$U \left| \psi_k \right\rangle = e^{iHt} \left| \psi_k \right\rangle$$ 

From the Taylor series for the exponent we have:

$$e^{iHt} \left| \psi_k \right\rangle = i t H \left| \psi_k \right\rangle - \frac{t^2}{2} H^2 \left| \psi_k \right\rangle + \frac{t^4}{4} H^4 \left| \psi_k \right\rangle - ...$$

And by using the Schrödinger equation $H \left| \psi_k \right\rangle = E_{k} \left| \psi_k \right\rangle$, we will obtain: 

$$e^{iHt} \left|\psi_k \right\rangle = e^{iE_{k}t} \left|\psi_k \right\rangle,$$ 

where $E_k$ is the eigenvalue of the corresponding $\left| \psi_k \right\rangle$ eigenstate. The goal of the algorithm is to find $E_k$s.

3) It easy to see that:

$$e^{iHt}\left|\psi_k \right\rangle = e^{iE_{k}t} \left|\psi_k \right\rangle  = e^{i 2 \pi \varphi_k} \left|\psi_k \right\rangle,$$

where $\varphi_k = E_{k}t / 2 \pi$ is the phase that the algorithm is capable to estimate. After estimating the phase it will be easy to estimate the corresponding eigenvalue:

$$ E_{k} = 2 \pi \varphi_k / t$$

Now let's see how the algorithm actually works. Firstly, we should import all the packages that we will use.

In [1]:
import numpy as np
from random import random
from qiskit import *
from qiskit.aqua.operators import WeightedPauliOperator, MatrixOperator
from qiskit.aqua.operators.op_converter import to_matrix_operator
from qiskit.aqua.utils.controlled_circuit import get_controlled_circuit

Then we are choosing a quantum computer simulator that we will use (namely ```'qasm_simulator'```). ```a``` and ```q``` are quantum registers. ```q``` register is the main register where $\left| \psi_k \right\rangle$ eigenstate will be encoded. $a$ register contains one ancillary qubit, where, after IPEA algorithm, the phase will be stored. In the end, only the ancillary qubit is measured and the measurement result will be stored in ```c``` classical register.

In [2]:
backend = BasicAer.get_backend('qasm_simulator')
q = QuantumRegister(1)
a = QuantumRegister(1)
c = ClassicalRegister(1)

```simple_hamiltonian_operator``` is a method that creates some simple (diagonal) Hamiltonian. In that case, the eigenstates coincide with the computational basis ($\left| 0 \right\rangle$ and $\left| 1 \right\rangle$). So the initialization of the eigenstate will be easy. In this tutorial, the eigenvalue of $\left| 1 \right\rangle$ is estimated.

In [3]:
def simple_hamiltonian_operator(E_1, E_2):
    """
    Creates 0.5*(E_1 + E_2)*I + 0.5*(E_1 - E_2)*Z pauli sum 
    that will be our "simple" Hamiltonian operator. The corresponding 
    matrix for Hamiltonian is [[E_1, 0], [0, E_2]].
    
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": 0.5 * (E_1 - E_2)}, "label": "Z"},
                   {"coeff": {"imag": 0.0, "real": 0.5 * (E_1 + E_2)}, "label": "I"}
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

Here the eigenvalue of $\left| 1 \right\rangle$ eigenstate is equal to $E_2$. Thus the algorithm should estimate the value of $E_2$. The Hamiltonian in matrix form looks like this:

$$H = \begin{pmatrix}
E_1 & 0\\
0 & E_2
\end{pmatrix} $$

This matrix should be exponentiated and applied on $\left| 1 \right\rangle$ eigenstate:

$$
e^{iHt}  \left|1 \right\rangle = \begin{pmatrix}
e^{i E_1 t} & 0\\
0 & e^{i E_2 t}
\end{pmatrix} 
\begin{pmatrix}
0\\
1
\end{pmatrix}= e^{i E_2 t}
\begin{pmatrix}
0\\
1
\end{pmatrix}$$

In [4]:
E_1, E_2 = (0, random())
print("We are going to estimate E_2 with IPEA algorithm = {}".format(E_2))

H = simple_hamiltonian_operator(E_1, E_2)
print("\nThe Hamiltonian operator in matrix form:")
print(to_matrix_operator(H).dense_matrix)

We are going to estimate E_2 with IPEA algorithm = 0.3339942860474068

The Hamiltonian operator in matrix form:
[[0.        +0.j 0.        +0.j]
 [0.        +0.j 0.33399429+0.j]]


The following code creates a circuit for $U = exp(-iHt)$. It is important to choose parameter $t$ for which $\varphi = E_k t / 2 \pi < 1$. If, for example $\varphi = 1.1$, then the $2 \pi \varphi = 2 \pi + 2 \pi \cdot 0.1$ and the estimated value for the phase will only be $0.1 \ne \varphi$. Consequently the estimated energy $E_{k} = 2 \pi \varphi / t$ will be wrong. 

Note, that the ```evolve``` method adds, in some sense undesired **"-"** sign. Because of the **"-"** sign something should be changed in the algorithm. In the case of ```evolve``` method we will have  $exp(-i 2 \pi \varphi) = exp(i 2 \pi (1 - \varphi)) = exp(i 2 \pi \tilde{\varphi})$. Thus, the algorithm will estimate not $\varphi = 1 - \tilde{\varphi}$, but $\tilde{\varphi}$. For simplicity let's omit the tilde above the $\tilde{\varphi}$.

In [5]:
t = 1
H_circuit = H.evolve(evo_time=t, quantum_registers=q)
# in older versions then Qiskit Aqua 0.6.2 one need to delete the barrier
# that is added in the end of the circuit in order to avoid errors caused 
# by get_controlled_circuit method which in older versions doesn't recognize
# barriers. For older versions add the following code:
# H_circuit.data.__delitem__(-1)

The following code creates a controlled version of the ```H_circuit``` on ```q``` register. The control qubit is the ancillary qubit. This means that if the ancillary is in $\left| 0 \right\rangle$ state the ```H_circuit``` will not run on ```q``` register and if the ancillary is in $\left| 1 \right\rangle$ state the ```H_circuit``` will run on ```q``` register. Also, if the ancillary qubit is in the $\left|+ \right\rangle$ superposition state and the qubit in ```q``` register is in $\left|1 \right\rangle$ eigenstate we will have:

$$\left|+ \right\rangle_a \left|1 \right\rangle_q = \frac{1}{\sqrt{2}} (\left|0 \right\rangle_a + \left|1 \right\rangle_a)\left|1 \right\rangle_q$$

After ```control_H``` circuit the state of the qubits will looks like this:

$$\left( \left|0 \right\rangle_a + e^{-i 2 \pi \varphi} \left|1 \right\rangle_a \right) \left|1 \right\rangle_q$$

This way we can store the value of the phase in the ancillary qubit. To give you an idea of how we are going to extract the phase let's imagine that the phase is either $0$ or $\frac{1}{2}$. If the phase is $0$ the state of the ancillary qubit will be $\left|+\right\rangle_a$ state and if it is $\frac{1}{2}$ the state will be $\left|- \right\rangle_a$. After this we apply to the ancillary qubit the Hadamard gate. In this step, the $\left|+\right\rangle_a$ state will transform to the $\left|0\right\rangle_a$ state and the $\left|-\right\rangle_a$ state will transform to the $\left|0\right\rangle_a$ state. So, just by measuring the ancillary qubit one will know is the phase $0$ or $\frac{1}{2}$.

In [6]:
control_H = get_controlled_circuit(H_circuit, a[0])

For general case we will represent $\varphi$ in its fractional binary representation:

$$\varphi = 0.\varphi_0 \varphi_1 \varphi_2 \varphi_3... =  \frac{\varphi_0}{2} + \frac{\varphi_1}{2^2} + \frac{\varphi_2}{2^3} + \frac{\varphi_3}{2^4} + ...$$

where each $\varphi_k$ is a bit (it is equal to $0$ or $1$). Here we should do the iterations. For each iteration, the algorithm is going to estimate one $\varphi_k$ bit at a time. The estimation is starting from the last bit. For example let's find $\varphi = 0.11$ phase. Notice, that after ```cantrol_H``` we will have the following exponent $e^{i 2 \pi 0.11}$. After applying ```cantrol_H``` twice one will obtain two exponents $e^{i 2 \pi 0.11} \cdot e^{i 2 \pi 0.11} = e^{i 2 \pi 0.1}$, because $2 \cdot 0.11 = 2 \cdot (\frac{1}{2} + \frac{1}{4}) = (1 + \frac{1}{2}) = 1.1$. This way we come to the simple example explained above ($\varphi$ is equal to $0$ or $\frac{1}{2}$). So it will be easy to estimate the second bit from $\varphi = 0.11$. At the second iteration the ```cantrol_H``` is applied once ($e^{i 2 \pi 0.11}$). Then the estimated second bit $\varphi_1$ is subtracted from the phase with a phase gate. In qiskit this action corresponds to the gate ```u1(-phase_shift, a[0])```, where ```phase_shift``` contains all the estimated bits. This way at each iteration we will come to the simple phase case ($0$ or $\frac{1}{2}$ case).

The following ```for``` loop iteratively estimates the phase.

In [7]:
num_bits_estimate = 10
phase = 0
for k_precision in reversed(range(num_bits_estimate)):
    # Create a Quantum Circuit acting on the q register
    k_circ = QuantumCircuit(q, a, c)

    # (1) |1> eigenstate initialization
    k_circ.x(q[0])

    # (2) Initial Hadamard gate applied on ancillary qubit.
    k_circ.h(a[0])

    # (3) The control Hamiltonian applied on the qubits where control qubit is the ancillary qubit.
    for order in range(2 ** k_precision):
        k_circ += control_H

    # (4) The phase gate and final Hadamard gate on ancillary qubit.
    phase_shift = 2 * np.pi * phase * 2 ** k_precision
    k_circ.u1(-phase_shift, a[0])
    k_circ.h(a[0])

    # (5) Measurement of ancillary qubit (findig the bit)
    k_circ.measure(a[0], c[0])

    # (6) executing on Quantum Computer and finding a bit from the phase
    job = execute(k_circ, backend, shots=1000)
    result = job.result()
    counts = result.get_counts()
    value = int(max(counts, key=counts.get))

    # (7) phase after this iteration
    phase += value / 2 ** (k_precision + 1)

Finally we are estimating the eigenvalue (energy) of the $\left| 1 \right\rangle$ eigenstate and printing the result alongside the true eigenvalue $E_2$.

In [8]:
eigenvalue = 2 * np.pi * (1 - phase) / t
print("Eigenvalue of Hamiltonian: {}".format(E_2))
print("Estimated eigenvalue of Hamiltonian: {}".format(eigenvalue))

Eigenvalue of Hamiltonian: 0.3339942860474068
Estimated eigenvalue of Hamiltonian: 0.3313398501832985


We omitted one step of the algorithm. That is the Trotter decomposition step. For some Hamiltonian $H = h_1 + h_2 + h_3$ we need to exponentiate it to obtain $U = e^{i H t}$ unitary operator. Actually with this algorithm (namely when we are constructing a circuit with ```evolve``` method) we obtain $\tilde{U} = e^{i h_1 t} \cdot e^{i h_2 t} \cdot e^{i h_3 t}$. Actually, $e^{iHt} \ne  e^{i h_1 t} \cdot e^{i h_2 t} \cdot e^{i h_3 t}$. To obtain more or less valid $U$ operator we should do Trotter decomposition:

$$ U = (e^{i h_1 \frac{t}{N}} \cdot e^{i h_2 \frac{t}{N}} \cdot e^{i h_3 \frac{t}{N}})^N + O(\frac{t^2}{N}),$$

where $N$ is the Trotter number [[2](https://www.tandfonline.com/doi/abs/10.1080/00268976.2011.552441)]. By increasing $N$ it is possible to obtain better representation for $U$ operator. Because our example of the Hamiltonian was simple (2x2 diagonal matrix) a reasonable result was obtained without doing this step.

**Final notes**: Note that instead of $E_2$ we can estimate $E_1$. For that we should prepare our qubit not in $\left| 1 \right\rangle$ state, but in $\left| 0 \right\rangle$ and estimate its eigenvalue (aka $E_1$). Here comes some problem that I didn't manage to solve. This code actually estimates $E_2 - E_1$ instead of $E_2$ and $E_1 = 0$ instead of $E_1$. This could be because ```evolve``` method doesn't work properly. It should construct a circuit that corresponds to $e^{-iHt}$, but instead, it creates a circuit for to $e^{-iH^*t}$, where $H^* = \begin{pmatrix}
0 & 0\\
0 & E_2 - E_1
\end{pmatrix}$. Currently, I am trying to solve this problem, which may be caused by a qiskit bug in ```evolve``` method or something different.

[1] [Quantum phase estimation algorithm](https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm)

[2]  [J.D. Whitfield,  J. Biamonte  and  A. Aspuru-Guzik,
Molecular Physics, "Simulation of electronic structure Hamiltonians using
quantum computers" (2011)](https://www.tandfonline.com/doi/abs/10.1080/00268976.2011.552441)